In [13]:
!pip install pandas_datareader
!pip install yfinance
!pip install arch


In [14]:
!pip install movingAverage

  ERROR: Could not find a version that satisfies the requirement moving (from versions: none)
ERROR: No matching distribution found for moving


In [11]:
import datetime as dt
import yfinance  as yf
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import re
import statsmodels.api as sm
import movingAverage as ma


from pandas_datareader import data as web
from arch.unitroot import ADF
from statsmodels.tsa.stattools import adfuller
from matplotlib.dates import DateFormatter, WeekdayLocator,\
    DayLocator, MONDAY,date2num
from mpl_finance import  candlestick_ohlc

from matplotlib.font_manager import FontProperties
font = FontProperties(fname=r"c:\windows\fonts\msjh.ttc", size=14, weight='normal', style='normal') # 步驟二

ModuleNotFoundError: No module named 'movingAverage'

In [ ]:
## 函數
class PairTrading:
    def SSD(self,priceX,priceY):
        if priceX is None or priceY is None:
            print('缺少價格序列.')
        standardX=priceX/priceX[0]
        standardY=priceY/priceY[0]
        SSD=np.sum((standardY-standardX)**2)
        return(SSD)
    def SSDSpread(self,priceX,priceY):
        if priceX is None or priceY is None:
            print('缺少價格序列.')
        standardX=priceX/priceX[0]
        standardY=priceY/priceY[0]
        spread=standardY-standardX
        return(spread)
    def cointegration(self,priceX,priceY):
        if priceX is None or priceY is None:
            print('缺少價格序列.')
        priceX=np.log(priceX)
        priceY=np.log(priceY)
        results=sm.OLS(priceY,sm.add_constant(priceX)).fit()
        resid=results.resid
        adfSpread=ADF(resid)
        if adfSpread.pvalue>=0.05:
            print('''交易價格不具有協整關係.
            P-value of ADF test: %f
            Coefficients of regression:
            Intercept: %f
            Beta: %f
             ''' % (adfSpread.pvalue, results.params[0], results.params[1]))
            return(None)
        else:
            print('''交易價格具有協整關係.
            P-value of ADF test: %f
            Coefficients of regression:
            Intercept: %f
            Beta: %f
             ''' % (adfSpread.pvalue, results.params[0], results.params[1]))
            return(results.params[0], results.params[1])
    def CointegrationSpread(self,priceX,priceY,
                            formStart,formEnd,tradeStart,tradeEnd):
        if priceX is None or priceY is None:
            print('缺少價格序列.')
        formX=priceX[formStart:formEnd]
        formY=priceY[formStart:formEnd]
        tradeX=priceX[tradeStart:tradeEnd]
        tradeY=priceY[tradeStart:tradeEnd]
        coefficients=self.cointegration(formX,formY)
        if coefficients is None:
                print('未形成協整關係,無法配對.')
        else:
            spread=(np.log(tradeY)
            -coefficients[0]-coefficients[1]*np.log(tradeX))
            return(spread)
    def calBound(self,priceX,priceY,method,formStart,formEnd,width=1.5):
        formX=priceX[formStart:formEnd]
        formY=priceY[formStart:formEnd]
        if method=='SSD':
            spread=self.SSDSpread(formX,formY)            
            mu=np.mean(spread)
            sd=np.std(spread)
            UpperBound=mu+width*sd
            LowerBound=mu-width*sd
            return(UpperBound,LowerBound)
        elif method=='Cointegration':
            spread=self.CointegrationSpread(priceX,priceY,formStart,formEnd,
                 formStart,formEnd)
            mu=np.mean(spread)
            sd=np.std(spread)
            UpperBound=mu+width*sd
            LowerBound=mu-width*sd
            return(UpperBound,LowerBound)
        else:
            print('不存在該方法. 請選擇"SSD"或是"Cointegration".')
                
                
                
                
                

# 布林通道函數
def upbreak(tsLine,tsRefLine):
    n=min(len(tsLine),len(tsRefLine))
    tsLine=tsLine[-n:]
    tsRefLine=tsRefLine[-n:]
    signal=pd.Series(0,index=tsLine.index)
    for i in range(1,len(tsLine)):
        if all([tsLine[i]>tsRefLine[i],tsLine[i-1]<tsRefLine[i-1]]):
            signal[i]=1
    return(signal)

def downbreak(tsLine,tsRefLine):
    n=min(len(tsLine),len(tsRefLine))
    tsLine=tsLine[-n:]
    tsRefLine=tsRefLine[-n:]
    signal=pd.Series(0,index=tsLine.index)
    for i in range(1,len(tsLine)):
        if all([tsLine[i]<tsRefLine[i],tsLine[i-1]>tsRefLine[i-1]]):
            signal[i]=1
    return(signal)

# 布林通道
#BBands
def bbands(tsPrice,period=20,times=2):
    upBBand=pd.Series(0.0,index=tsPrice.index)
    midBBand=pd.Series(0.0,index=tsPrice.index)
    downBBand=pd.Series(0.0,index=tsPrice.index)
    sigma=pd.Series(0.0,index=tsPrice.index)
    for i in range(period-1,len(tsPrice)):
        midBBand[i]=np.nanmean(tsPrice[i-(period-1):(i+1)]) 
        sigma[i]=np.nanstd(tsPrice[i-(period-1):(i+1)])
        upBBand[i]=midBBand[i]+times*sigma[i]
        downBBand[i]=midBBand[i]-times*sigma[i]
    BBands=pd.DataFrame({'upBBand':upBBand[(period-1):],\
                         'midBBand':midBBand[(period-1):],\
                         'downBBand':downBBand[(period-1):],\
                         'sigma':sigma[(period-1):]})
    return(BBands)



def candleLinePlots(candleData, candleTitle='a', **kwargs):
    Date = [date2num(date) for date in candleData.index]
    candleData.loc[:,'Date'] = Date
    listData = []    
    for i in range(len(candleData)):
        a = [candleData.Date[i],\
            candleData.Open[i],candleData.High[i],\
            candleData.Low[i],candleData.Close[i]]
        listData.append(a)
    # 如 果 不 定 長 參 數 無 取 值 ， 只 畫 蠟 燭 圖
    fig = plt.figure(figsize=(18,8))
    ax = plt.subplot()    
    # 如 果 不 定 長 參 數 有 值 ， 則 分 成 兩 個 子 圖
    ax = plt.subplot(111)
    
    ##### 布林通道畫圖

    mondays = WeekdayLocator(MONDAY)
    weekFormatter = DateFormatter('%y %b %d')
    ax.xaxis.set_major_locator(mondays)
    ax.xaxis.set_minor_locator(DayLocator())
    ax.xaxis.set_major_formatter(weekFormatter)
    plt.sca(ax)
    
    candlestick_ohlc(ax,listData, width=0.7,\
                     colorup='r',colordown='g')
    ax.set_title(candleTitle, fontproperties = font) ##
    plt.setp(ax.get_xticklabels(),\
             rotation=20,\
             horizontalalignment='center')
    
    Close = candleData.Close
    BBands=bbands(Close,20,2)
    BBdown = BBands[['downBBand']]
    BBup = BBands[['upBBand']]
    plt.plot(BBdown,label='downBBand')
    plt.plot(BBup,label='upBBand')
    plt.legend(loc='best')
    ax.autoscale_view()
    
def MACD(start,end,df_close,title):
    Close = df_close
    #MACD
    DIF=ma.ewmaCal(Close,12,2/(1+12))\
           -ma.ewmaCal(Close,26,2/(1+26))
    DIF.tail(n=3)

    DEA=ma.ewmaCal(DIF.dropna(),9,2/(1+9))
    DEA.tail()

    MACD=DIF-DEA
    MACD.tail(n=3)

    macddata=pd.DataFrame()
    macddata['DIF']= DIF[start:end]
    macddata['DEA']= DEA[start:end]
    macddata['MACD']= MACD[start:end]
    
    fig = plt.figure(figsize=(18,2))
    #plt.subplot(211)
    plt.plot(DIF[start:end],\
          label="DIF",color='k')
    plt.plot(DEA[start:end], label="DEA",\
            color='b',linestyle='dashed')
    #plt.title('信號線DIF与DEA', FontProperties = font)
    #plt.legend()
    #plt.subplot(212)
    plt.bar(MACD[start:end].index,\
            height=MACD[start:end],\
            label='MACD',color='r')
    plt.title(title, FontProperties = font)
    plt.legend()  
    

def rsi(price,period=6,period_1=24,xlabel='stock'):
    closeDif=(price-price.shift(1)).dropna() # 將第一個值變為NA & 刪除有NA的資料筆
    upPrc=pd.Series(0,index=closeDif.index)  # 以日期為index, 數值為0的資料集 ??
    upPrc[closeDif>0]=closeDif[closeDif>0]   # 將 > 0 的數值放入
    downPrc=pd.Series(0,index=closeDif.index)
    downPrc[closeDif<0]=-closeDif[closeDif<0]
    rsi=[]
    for i in range(period,len(upPrc)+1):
        up_mean = np.mean(upPrc.values[(i-period):i],\
        dtype=np.float32)  # up 六天平均值
        
        up_down = np.mean(downPrc.values[(i-period):i],\
        dtype=np.float32)  # down 六天平均值
        
        rsi.append(100*up_mean/(up_mean+up_down)) # rsi6 = (100*up) / (up+down) 
    rsi=pd.Series(rsi,index=closeDif.index[(period-1):])
    fig = plt.figure(figsize=(18,2))
    plt.plot(rsi)
    plt.title('RSI6的超買線和超賣線', fontproperties = font)
    plt.xlabel(xlabel, FontProperties=font)
    plt.ylim(-10,110)
    plt.axhline(y=80,color='red')
    plt.axhline(y=20,color='red')
    plt.show()
    #---------------------------------------------------------------------------------------------------
    # period_1
    closeDif_1=(price-price.shift(1)).dropna() # 將第一個值變為NA & 刪除有NA的資料筆
    upPrc_1=pd.Series(0,index=closeDif_1.index)  # 以日期為index, 數值為0的資料集 ??
    upPrc_1[closeDif_1>0]=closeDif_1[closeDif_1>0]   # 將 > 0 的數值放入
    downPrc_1=pd.Series(0,index=closeDif_1.index)
    downPrc_1[closeDif_1<0]=-closeDif_1[closeDif_1<0]
    rsi_1=[]
    for i in range(period_1,len(upPrc_1)+1):
        up_mean_1 = np.mean(upPrc_1.values[(i-period_1):i],\
        dtype=np.float32)  # up 六天平均值
        
        up_down_1 = np.mean(downPrc_1.values[(i-period_1):i],\
        dtype=np.float32)  # down 六天平均值
        
        rsi_1.append(100*up_mean_1/(up_mean_1+up_down_1)) # rsi6 = (100*up) / (up+down) 
    rsi_1=pd.Series(rsi_1,index=closeDif_1.index[(period_1-1):])
    
    fig = plt.figure(figsize=(18,2))
    plt.plot(rsi,label='Rsi6')
    plt.plot(rsi_1,
             label='Rsi24',color='red',\
             linestyle='dashed')
    plt.title('RSI黃金交叉和死亡交叉', fontproperties = font)
    plt.xlabel(xlabel, FontProperties=font)
    plt.ylim(-10,110)
    plt.legend()
    plt.show()
    
def KDJ(df,start,end,xlabel):
    Close=df.Close
    High=df.High
    Low=df.Low
    ndate=len(Close)
    # 計算9天未成熟隨機指標 RSV 的值
    RSV=[]
    for j in range(8,ndate):
        periodHigh=np.max(High[(j-8):(j+1)])
        periodLow=np.min(Low[(j-8):(j+1)])
        RSV.append(100*(Close[j]-periodLow)\
               /(periodHigh-periodLow))

    RSV=pd.Series(RSV,index=Close.index[8:])
    RSV.name='RSV'
    # 計算 K D 值
    K=[50]
    D=[50]

    for i in range(len(RSV)):
        KValue = (2/3)*K[-1] + (RSV[i]/3)
        DValue = (2/3)*D[-1] + KValue/3
        K.append(KValue)
        D.append(DValue)
        
    K=pd.Series(K[1:],index=RSV.index)
    K.name='KValue'
    D=pd.Series(D[1:],index=RSV.index)
    D.name='DValue'
    J=3*K-2*D
    J.name='JValue'
    fig = plt.figure(figsize=(18,2))
    plt.title('RSV與KDJ', fontproperties = font)
    plt.xlabel(xlabel, FontProperties=font)
    plt.plot(RSV[start:end])
    #plt.plot(K[start:end],linestyle='dashed')
    #plt.plot(D[start:end],linestyle='-.')
    plt.plot(J[start:end],linestyle='--')
    plt.legend(loc='upper left') 

In [ ]:
# df_high 高價股資料
# df_low  低價股資料
# formStart 形成期-開始
# formEnd   形成期-結束
# tradeStart  交易期-開始
# tradeEnd    交易期-結束
def fit_stock(df_high,df_low,formStart='2018-01',formEnd='2018-12',tradeStart='2019-01',tradeEnd='2019-06',\
              h_title='high',l_title='low'):
    df_high['Date'] = df_high.index.strftime("%Y-%m-%d")
    df_low['Date'] = df_low.index.strftime("%Y-%m-%d")
    #========================================================================================================================
    #--------------------------------------------------------------------------------
    df_highf=df_high[formStart:formEnd]
    df_lowf=df_low[formStart:formEnd]
    pairf=pd.concat([df_high.Close,df_low.Close],axis = 1)  #合併 股票
    #---------------------------------------------------------------------------------------------- log 
    df_highflog=np.log(df_highf.Close) # 對高價股收盤價取log
    df_lowflog=np.log(df_lowf.Close) # 對低價股收盤價取log
    #---------------------------------------------------------------------------------------------- 一階差分
    df_ret_high=df_highflog.diff()[1:]  #對高價股log(收盤價)取一階差分
    df_ret_low=df_lowflog.diff()[1:]  #對高價股log(收盤價)取一階差分
    #---------------------------------------------------------------------------------------------- 計算殘差
    model=sm.OLS(df_lowflog,sm.add_constant(df_highflog))  #建模
    results=model.fit()
    alpha=results.params[0]  
    beta=results.params[1]
    spread=df_lowflog-beta*df_highflog-alpha
    #---------------------------------------------------------------------------------------------- 檢定是否具有協整關係
    priceA= df_high.Close # 4976佳凌科技
    priceB=df_low.Close  # 8215明碁材料
    priceAf=priceA[formStart:formEnd]
    priceBf=priceB[formStart:formEnd]
    priceAt=priceA[tradeStart:tradeEnd]
    priceBt=priceB[tradeStart:tradeEnd]
    
    pt=PairTrading()
    SSD=pt.SSD(priceAf,priceBf)
    SSDspread=pt.SSDSpread(priceAf,priceBf)
    coefficients=pt.cointegration(priceAf,priceBf)
    
    CoSpreadF=pt.CointegrationSpread(priceA,priceB,
             formStart,formEnd,formStart,formEnd)
    
    CoSpreadTr=pt.CointegrationSpread(priceA,priceB,formStart,formEnd,
            tradeStart,tradeEnd)
    
    bound=pt.calBound(priceA,priceB,'Cointegration',
      formStart,formEnd,width=1.2)
    
    ##########配對交易實測 ###############
    mu=np.mean(CoSpreadF)
    sd=np.std(CoSpreadF)
    
    #---------------------------------------------------------------------------------------------- 
    
    
    
    
    #========================================================================================================================
    #========================================================================================================================
    fig = plt.figure(figsize=(18,10))
    ax1 = plt.subplot(511)
    fig = plt.figure(figsize=(18,10))
    ax2 = plt.subplot(512)
    fig = plt.figure(figsize=(18,10))
    ax3 = plt.subplot(513)
    fig = plt.figure(figsize=(18,10))
    ax4 = plt.subplot(514)
    fig = plt.figure(figsize=(20,30))
    ax5 = plt.subplot(515)
    
    #----------------------------------------------------------------------------------------------- 收盤價 畫圖
    plt.sca(ax1)
    plt.plot(pairf)
    plt.ylabel("股價", FontProperties=font)
    plt.legend(labels=['8454', '2913'],  loc='best')
    plt.title("富邦媒體 與 台灣農林 收盤價圖", FontProperties=font)
    #----------------------------------------------------------------------------------------------- log(收盤價) 畫圖
    #  兩隻配對股log 時序圖 
    plt.sca(ax2)
    plt.plot(df_highflog)
    plt.plot(df_lowflog)
    plt.legend(labels=['8454', '2913'],  loc='best')
    plt.title('富邦媒體 與 台灣農林--對數價格時序圖(形成期)', FontProperties = font) 
    #----------------------------------------------------------------------------------------------- 一階差分log(收盤價) 畫圖
    #  兩隻配對股log 一階差分 時序圖
    plt.sca(ax3)
    plt.plot(df_ret_high)
    plt.plot(df_ret_low)
    plt.legend(labels=['8454', '2913'],  loc='best')
    plt.title('富邦媒體 與 台灣農林---對數價格一階差分時序圖(形成期)', FontProperties = font) 
    #----------------------------------------------------------------------------------------------- 殘差 畫圖
    plt.sca(ax4)
    spread.plot()
    plt.title('富邦媒體8454 與 台灣農林2913 的殘差序列 (形成期)', FontProperties = font) #回歸殘差
    #----------------------------------------------------------------------------------------------- 交易期價差序列(協整配對) 畫圖   
    #設定交易期
    plt.sca(ax5)
    CoSpreadTr.plot()
    plt.title('交易期價差序列(協整配對)', FontProperties = font)
    plt.axhline(y=mu,color='black')
    plt.axhline(y=mu+0.2*sd,color='blue',ls='-',lw=2)
    plt.axhline(y=mu-0.2*sd,color='blue',ls='-',lw=2)
    plt.axhline(y=mu+1.5*sd,color='green',ls='--',lw=2.5)
    plt.axhline(y=mu-1.5*sd,color='green',ls='--',lw=2.5)
    plt.axhline(y=mu+2.5*sd,color='red',ls='-.',lw=3) 
    plt.axhline(y=mu-2.5*sd,color='red',ls='-.',lw=3) 
    
    #----------------------------------------------------------------------------------------------- 布林通道 & K線圖
    candleLinePlots(candleData=df_high[formStart:formEnd], candleTitle='8454--2018年')
    candleLinePlots(candleData=df_low[formStart:formEnd], candleTitle='2913--2018年')
    #----------------------------------------------------------------------------------------------- MACD
    MACD('2018-01','2018-12',df_high.Close,h_title) # 4976
    MACD('2018-01','2018-12',df_low.Close,l_title) # 8215
    #----------------------------------------------------------------------------------------------- RSI 6日均線 & 24日均線 &交叉
    rsi(df_high.Close,period=6,period_1=24,xlabel=h_title)
    rsi(df_low.Close,period=6,period_1=24,xlabel=l_title)
    #----------------------------------------------------------------------------------------------- KDJ與RSV
    KDJ(df_high,formStart,formEnd,xlabel=h_title)
    KDJ(df_low,formStart,formEnd,xlabel=l_title)
    
    
    
    

In [ ]:
startTime = '2018-01-01'
endTime = '2019-11-01'
# 4976  8215
df_8454 = web.get_data_yahoo('8454.TW',startTime, endTime) #高價股  佳凌科技
df_2913 = web.get_data_yahoo('2913.TW',startTime, endTime) #低價股  明基材料

In [2]:
fit_stock(df_8454,df_2913,formStart='2018-01',formEnd='2018-12',tradeStart='2019-01',tradeEnd='2019-06',\
          h_title='8454',l_title='2913')

NameError: name 'fit_stock' is not defined